__Assignment 6__

6) Compare the average monthly temperature (find the difference) in the period 1950-2014 for all stations in Östergotland with long-term monthly averages in the period of 1950-1980. Make a plot of your results.

__HINT:__ The __first step__ is to find the monthly averages for each station. In the __next step__, you can average over all stations to acquire the average temperature for a specific year and month. This RDD/Data Frame can be used to compute the long-term average by averaging over all the years in the interval.

The output should contain the following information:
__Year, month, difference__

For each question include the following data in the report and sort it as shown:
- year, month, difference ORDER BY year DESC, month DESC

In [172]:
# create the setup
from pyspark import SparkContext
from pyspark.sql import SQLContext, Row  
from pyspark.sql import functions as F
sc =SparkContext()
sqlContext=SQLContext(sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-1-ec3c098af780>:5 

In [173]:
from pyspark.sql.functions import col, round, upper
from  pyspark.sql.functions import abs

In [174]:
path_unifolder_temp_tiny = "/Users/phillipholscher/OneDrive - Linköpings universitet/LiU/Statistics and Machine Learning/Spring19T2/Big Data Analytics/Labs/Lab 2/temperature-readings-tiny.csv"
path_unifolder_stations = "/Users/phillipholscher/OneDrive - Linköpings universitet/LiU/Statistics and Machine Learning/Spring19T2/Big Data Analytics/Labs/Lab 2/stations-Ostergotland.csv"


In [175]:
# read and create rdd, this needs to be tranfsormed to data frame later on
temps = sc.textFile(path_unifolder_temp_tiny)
parts_temps = temps.map(lambda l:l.split(";"))
tempReadings = parts_temps.map(lambda p: Row(station=p[0],  date=p[1], year=p[1].split("-")[0], month=p[1].split("-")[1], day=p[1].split("-")[2], time=p[2],  Temp=float(p[3]), quality=p[4]))
tempReadings.take(10)


[Row(Temp=6.8, date='2013-11-01', day='01', month='11', quality='G', station='102170', time='06:00:00', year='2013'),
 Row(Temp=3.8, date='2013-11-01', day='01', month='11', quality='G', station='102170', time='18:00:00', year='2013'),
 Row(Temp=5.8, date='2013-11-02', day='02', month='11', quality='G', station='102170', time='06:00:00', year='2013'),
 Row(Temp=-1.1, date='2013-11-02', day='02', month='11', quality='G', station='102170', time='18:00:00', year='2013'),
 Row(Temp=-0.2, date='2013-11-03', day='03', month='11', quality='G', station='102170', time='06:00:00', year='2013'),
 Row(Temp=5.6, date='2013-11-03', day='03', month='11', quality='G', station='102170', time='18:00:00', year='2013'),
 Row(Temp=6.5, date='2013-11-04', day='04', month='11', quality='G', station='102170', time='06:00:00', year='2013'),
 Row(Temp=5.1, date='2013-11-04', day='04', month='11', quality='G', station='102170', time='18:00:00', year='2013'),
 Row(Temp=4.2, date='2013-11-05', day='05', month='11'

In [176]:
# create a data frame from rdd
schemaTempReadings =  sqlContext.createDataFrame(tempReadings)
schemaTempReadings.registerTempTable("tempReadings")
schemaTempReadings.show(10)

+----+----------+---+-----+-------+-------+--------+----+
|Temp|      date|day|month|quality|station|    time|year|
+----+----------+---+-----+-------+-------+--------+----+
| 6.8|2013-11-01| 01|   11|      G| 102170|06:00:00|2013|
| 3.8|2013-11-01| 01|   11|      G| 102170|18:00:00|2013|
| 5.8|2013-11-02| 02|   11|      G| 102170|06:00:00|2013|
|-1.1|2013-11-02| 02|   11|      G| 102170|18:00:00|2013|
|-0.2|2013-11-03| 03|   11|      G| 102170|06:00:00|2013|
| 5.6|2013-11-03| 03|   11|      G| 102170|18:00:00|2013|
| 6.5|2013-11-04| 04|   11|      G| 102170|06:00:00|2013|
| 5.1|2013-11-04| 04|   11|      G| 102170|18:00:00|2013|
| 4.2|2013-11-05| 05|   11|      G| 102170|06:00:00|2013|
| 3.2|2013-11-05| 05|   11|      G| 102170|18:00:00|2013|
+----+----------+---+-----+-------+-------+--------+----+
only showing top 10 rows



In [177]:
# read and create rdd, this needs to be tranfsormed to data frame later on
stations = sc.textFile(path_unifolder_stations)
# create DataFrame from RDD
parts_stations = stations.map(lambda a: a.split(';'))
schemaStationsOestergotland = parts_stations.map(lambda x: Row(station = x[0], name = x[1]))
schemaStationsOestergotland = sqlContext.createDataFrame(schemaStationsOestergotland)

schemaStationsOestergotland.show(100) # this list is with all stations

+--------------------+-------+
|                name|station|
+--------------------+-------+
|             Örberga|  84260|
|           Västra Ny|  85630|
|         Kettstaka A|  85460|
|            Godegård|  85450|
|          Zinkgruvan|  85490|
|          Kärnskogen|  85650|
|            Kvarn Mo|  85390|
|            Finspång|  85410|
|          Simonstorp|  86470|
|         Stavsjö Aut|  86440|
|Kolmården-Strömsf...|  86420|
|     Norrköping-SMHI|  86340|
|Norrköping-Kungsä...|  86350|
|          Norrköping|  86360|
|    Norrköping-Sörby|  86370|
|               Holma|  86200|
|            Marviken|  86330|
|          Harstena A|  87140|
|            Harstena|  87150|
|        Valdemarsvik|  86130|
|             Falerum|  86090|
|            Skärkind|  85280|
|           Linköping|  85250|
|           Malmslätt|  85240|
|          Västerlösa|  85270|
|              Öjebro|  85220|
|           Herrberga|  85210|
|             Härsnäs|  85180|
|          Malexander|  85040|
|       

In [178]:
# join to filter for ostg stations
schemaTempReadings2 = schemaTempReadings.join(schemaStationsOestergotland, 'station', 'inner')
schemaTempReadings2.show(10)
# the result would be empty - there for we continue without filter by stations

+-------+----+----+---+-----+-------+----+----+----+
|station|Temp|date|day|month|quality|time|year|name|
+-------+----+----+---+-----+-------+----+----+----+
+-------+----+----+---+-----+-------+----+----+----+



__First step__ is to find the monthly averages for each station

In [179]:
# filter for given time interval
tempReadings_filtered_2014 = schemaTempReadings\
.select(["station", "year", "month", "day", "Temp"])\
.filter((schemaTempReadings["year"] >= 1950) & (schemaTempReadings["year"] <= 2014))

tempReadings_filtered_2014.show(10)

+-------+----+-----+---+----+
|station|year|month|day|Temp|
+-------+----+-----+---+----+
| 102170|2013|   11| 01| 6.8|
| 102170|2013|   11| 01| 3.8|
| 102170|2013|   11| 02| 5.8|
| 102170|2013|   11| 02|-1.1|
| 102170|2013|   11| 03|-0.2|
| 102170|2013|   11| 03| 5.6|
| 102170|2013|   11| 04| 6.5|
| 102170|2013|   11| 04| 5.1|
| 102170|2013|   11| 05| 4.2|
| 102170|2013|   11| 05| 3.2|
+-------+----+-----+---+----+
only showing top 10 rows



In [180]:
# to calculate the monthly avg for each station do we need the max and min temp for each month and station
schemaTempReadings_max_min_2014 = tempReadings_filtered_2014.groupBy("year", "month", "station")\
.agg(F.max("Temp").alias("Temp_max"),F.min("Temp").alias("Temp_min"))

schemaTempReadings_max_min_2014.show(10)

+----+-----+-------+--------+--------+
|year|month|station|Temp_max|Temp_min|
+----+-----+-------+--------+--------+
|1959|   08| 102190|    28.2|     5.3|
|1959|   12| 102190|     5.0|   -18.8|
|1956|   01| 102190|     4.0|   -30.0|
|1959|   07| 102190|    27.4|     4.8|
|2014|   01| 102170|     6.6|   -19.6|
|1956|   06| 102190|    26.0|     3.8|
|1956|   08| 102190|    19.1|     3.9|
|1958|   02| 102190|     2.9|   -27.9|
|1958|   03| 102190|     5.7|   -20.0|
|1960|   07| 102190|    25.0|     6.0|
+----+-----+-------+--------+--------+
only showing top 10 rows



In [181]:
# function to calculate the avg temp as in the exercise asked for
def avg_temp(max_temp, min_temp):
    avg = (max_temp + min_temp)/2.0
    return avg

In [182]:
# compute the avg monthly temp for each station
schemaTempReadings_avg_2014 = schemaTempReadings_max_min_2014\
.withColumn("Temp_avg", 
            round(avg_temp(schemaTempReadings_max_min_2014["Temp_max"],
                           schemaTempReadings_max_min_2014["Temp_min"]),1))
schemaTempReadings_avg_2014.show(10)

+----+-----+-------+--------+--------+--------+
|year|month|station|Temp_max|Temp_min|Temp_avg|
+----+-----+-------+--------+--------+--------+
|1959|   08| 102190|    28.2|     5.3|    16.8|
|1959|   12| 102190|     5.0|   -18.8|    -6.9|
|1956|   01| 102190|     4.0|   -30.0|   -13.0|
|1959|   07| 102190|    27.4|     4.8|    16.1|
|2014|   01| 102170|     6.6|   -19.6|    -6.5|
|1956|   06| 102190|    26.0|     3.8|    14.9|
|1956|   08| 102190|    19.1|     3.9|    11.5|
|1958|   02| 102190|     2.9|   -27.9|   -12.5|
|1958|   03| 102190|     5.7|   -20.0|    -7.2|
|1960|   07| 102190|    25.0|     6.0|    15.5|
+----+-----+-------+--------+--------+--------+
only showing top 10 rows



In [183]:
# sort the avg temp 
schemaTempReadings_avg_ordered_2014 = schemaTempReadings_avg_2014.select(["year", "month", "Temp_avg"])\
.orderBy("Temp_avg", ascending = False)

schemaTempReadings_avg_ordered_2014.show(10)


+----+-----+--------+
|year|month|Temp_avg|
+----+-----+--------+
|2014|   07|    20.4|
|1960|   06|    17.1|
|1959|   08|    16.8|
|1959|   07|    16.1|
|1958|   07|    16.1|
|1960|   07|    15.5|
|1960|   08|    15.4|
|1956|   07|    15.3|
|2014|   06|    15.2|
|1957|   07|    15.2|
+----+-----+--------+
only showing top 10 rows



__Second step__  average over all stations

In [184]:
# compute the long term monthly avg

# filter >= 1950 & <= 1980
long_term_month_avg_filterd = schemaTempReadings_avg_ordered_2014.filter((schemaTempReadings_avg_ordered_2014["year"] >= 1950) &
                                                     (schemaTempReadings_avg_ordered_2014["year"] <= 1980))
long_term_month_avg_filterd.show(10) 


+----+-----+--------+
|year|month|Temp_avg|
+----+-----+--------+
|1960|   06|    17.1|
|1959|   08|    16.8|
|1959|   07|    16.1|
|1958|   07|    16.1|
|1960|   07|    15.5|
|1960|   08|    15.4|
|1956|   07|    15.3|
|1957|   07|    15.2|
|1956|   06|    14.9|
|1959|   06|    14.7|
+----+-----+--------+
only showing top 10 rows



In [185]:
# avg over all month
long_term_month_avg = long_term_month_avg_filterd\
.groupBy("month").agg(round(F.avg("Temp_avg"),1).alias("LT_avg_monthly"))

long_term_month_avg.show(10)

+-----+--------------+
|month|LT_avg_monthly|
+-----+--------------+
|   07|          15.6|
|   11|          -2.4|
|   01|          -9.4|
|   09|           9.5|
|   05|           9.9|
|   08|          14.3|
|   03|          -2.5|
|   02|          -9.3|
|   06|          14.0|
|   10|           3.6|
+-----+--------------+
only showing top 10 rows



In [186]:
# function to compute the differences
def differences(Temp_avg, montly_avg):
    return abs(Temp_avg - montly_avg)

In [187]:
# to compute the differences we have to join the data frames first
temp_differ = schemaTempReadings_avg_ordered_2014.join(long_term_month_avg, "month", "inner")

temp_differ.show(10)


+-----+----+--------+--------------+
|month|year|Temp_avg|LT_avg_monthly|
+-----+----+--------+--------------+
|   07|2014|    20.4|          15.6|
|   07|1959|    16.1|          15.6|
|   07|1958|    16.1|          15.6|
|   07|1960|    15.5|          15.6|
|   07|1956|    15.3|          15.6|
|   07|1957|    15.2|          15.6|
|   11|2014|     1.2|          -2.4|
|   11|1958|     0.4|          -2.4|
|   11|1959|    -0.9|          -2.4|
|   11|1960|    -1.4|          -2.4|
+-----+----+--------+--------------+
only showing top 10 rows



In [188]:
# compute the temp difference
temp_differ = temp_differ.withColumn("differences",
                                     round(differences(temp_differ["Temp_avg"],
                                                 temp_differ["LT_avg_monthly"]),1))\
.orderBy(["year", "month"], ascending = False)
temp_differ.show(10)

+-----+----+--------+--------------+-----------+
|month|year|Temp_avg|LT_avg_monthly|differences|
+-----+----+--------+--------------+-----------+
|   12|2014|    -9.4|          -6.5|        2.9|
|   11|2014|     1.2|          -2.4|        3.6|
|   10|2014|     5.1|           3.6|        1.5|
|   09|2014|     7.7|           9.5|        1.8|
|   08|2014|    14.8|          14.3|        0.5|
|   07|2014|    20.4|          15.6|        4.8|
|   06|2014|    15.2|          14.0|        1.2|
|   05|2014|    12.6|           9.9|        2.7|
|   04|2014|     5.2|           3.4|        1.8|
|   03|2014|     1.7|          -2.5|        4.2|
+-----+----+--------+--------------+-----------+
only showing top 10 rows



In [189]:
# show the required result
temp_differ_result = temp_differ.select("year", "month", "differences")

temp_differ_result.show(10)


+----+-----+-----------+
|year|month|differences|
+----+-----+-----------+
|2014|   12|        2.9|
|2014|   11|        3.6|
|2014|   10|        1.5|
|2014|   09|        1.8|
|2014|   08|        0.5|
|2014|   07|        4.8|
|2014|   06|        1.2|
|2014|   05|        2.7|
|2014|   04|        1.8|
|2014|   03|        4.2|
+----+-----+-----------+
only showing top 10 rows



__*Github version*__

In [ ]:
# read the temperature data
rdd = sc.textFile("/Users/phillipholscher/OneDrive - Linköpings universitet/LiU/Statistics and Machine Learning/Spring19T2/Big Data Analytics/Labs/Lab 2/temperature-readings-tiny.csv")
# create DataFrame from RDD
parts = rdd.map(lambda a: a.split(';'))
tempReadingsRow = parts.map(lambda x: (x[0], x[1], int(x[1].split("-")[0]), int(x[1].split("-")[1]), x[2], float(x[3]), x[4]))
tempReadingsString = ["station", "date", "year", "month", "time", "value", "quality"]
schemaTempReadings = sqlContext.createDataFrame(tempReadingsRow, tempReadingsString)

In [ ]:
# read the Ostergotland stations data
rdd = sc.textFile("/Users/phillipholscher/OneDrive - Linköpings universitet/LiU/Statistics and Machine Learning/Spring19T2/Big Data Analytics/Labs/Lab 2/stations-Ostergotland.csv")
# create DataFrame from RDD
parts = rdd.map(lambda a: a.split(';'))
statOstRow = parts.map(lambda x: (x[0], x[1]))
statOstString = ["station", "name"]
schemaStatOst = sqlContext.createDataFrame(statOstRow, statOstString)

In [ ]:
# join to filter for ostg stations
schemaTempReadings = schemaTempReadings.join(schemaStatOst, 'station', 'inner')
schemaTempReadings.show(10)

**long term monthly avg - version 2**

In [190]:
# compute the long term monthly avg

# filter >= 1950 & <= 1980
long_term_month_avg_filterd_version2 = schemaTempReadings.filter((schemaTempReadings["year"] >= 1950) &
                                                     (schemaTempReadings["year"] <= 1980))
long_term_month_avg_filterd_version2.show(10) 


+----+----------+---+-----+-------+-------+--------+----+
|Temp|      date|day|month|quality|station|    time|year|
+----+----------+---+-----+-------+-------+--------+----+
|14.5|1955-09-01| 01|   09|      Y| 102190|06:00:00|1955|
|19.8|1955-09-01| 01|   09|      Y| 102190|12:00:00|1955|
|15.2|1955-09-01| 01|   09|      Y| 102190|18:00:00|1955|
|11.6|1955-09-02| 02|   09|      Y| 102190|06:00:00|1955|
|15.8|1955-09-02| 02|   09|      Y| 102190|12:00:00|1955|
|12.3|1955-09-02| 02|   09|      Y| 102190|18:00:00|1955|
|11.5|1955-09-03| 03|   09|      Y| 102190|06:00:00|1955|
|14.5|1955-09-03| 03|   09|      Y| 102190|12:00:00|1955|
|13.4|1955-09-03| 03|   09|      Y| 102190|18:00:00|1955|
| 7.7|1955-09-04| 04|   09|      Y| 102190|06:00:00|1955|
+----+----------+---+-----+-------+-------+--------+----+
only showing top 10 rows



In [191]:
# avg over all month
long_term_month_avg_version2 = long_term_month_avg_filterd_version2\
.groupBy('year','month','station').agg(round(F.avg('Temp'),1).alias('Temp_avg'))

long_term_month_avg_version2.show(10)

+----+-----+-------+--------+
|year|month|station|Temp_avg|
+----+-----+-------+--------+
|1959|   08| 102190|    15.7|
|1959|   12| 102190|    -2.4|
|1956|   01| 102190|    -9.3|
|1959|   07| 102190|    17.1|
|1956|   06| 102190|    13.0|
|1956|   08| 102190|    11.5|
|1958|   02| 102190|    -9.7|
|1958|   03| 102190|    -6.3|
|1960|   07| 102190|    14.2|
|1958|   08| 102190|    13.2|
+----+-----+-------+--------+
only showing top 10 rows



In [192]:
long_term_month_avg_version2 = long_term_month_avg_version2\
.groupBy('month').agg(round(F.avg('Temp_avg'),1).alias("Temp_avg_monthly"))\
.orderBy("month", ascending = False)

long_term_month_avg_version2.show(12)

+-----+----------------+
|month|Temp_avg_monthly|
+-----+----------------+
|   12|            -4.0|
|   11|            -0.4|
|   10|             4.5|
|   09|             9.5|
|   08|            13.5|
|   07|            15.6|
|   06|            13.9|
|   05|             9.6|
|   04|             2.9|
|   03|            -2.0|
|   02|            -6.6|
|   01|            -7.3|
+-----+----------------+



In [193]:
# to compute the differences we have to join the data frames first
temp_differ_v2 = schemaTempReadings_avg_ordered_2014.join(long_term_month_avg_version2, "month", "inner")


# compute the temp difference
temp_differ_v2 = temp_differ_v2.withColumn("differences",
                                     round(differences(temp_differ_v2["Temp_avg"],
                                                 temp_differ_v2["Temp_avg_monthly"]),1))\
.orderBy(["year", "month"], ascending = False)

# show the required result
temp_differ_result_v2 = temp_differ_v2.select("year", "month", "differences")

temp_differ_result_v2.show(10)


+----+-----+-----------+
|year|month|differences|
+----+-----+-----------+
|2014|   12|        5.4|
|2014|   11|        1.6|
|2014|   10|        0.6|
|2014|   09|        1.8|
|2014|   08|        1.3|
|2014|   07|        4.8|
|2014|   06|        1.3|
|2014|   05|        3.0|
|2014|   04|        2.3|
|2014|   03|        3.7|
+----+-----+-----------+
only showing top 10 rows

